<a href="https://colab.research.google.com/github/talalba/Instagram_Like_Predictor/blob/master/lid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import the necessary packages
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import importlib
import itertools
import nltk
from nltk import ngrams
from nltk import word_tokenize
import string
import collections
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [272]:
# load the data
directoryPath = '/content/drive/My Drive/lid/'
# df = pd.read_csv(directoryPath+'formatted_data.csv',sep=';',encoding='latin1')
df = pd.read_csv(directoryPath+'formatted_data.csv',sep=';')
df.shape

# function to remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

# function to remove whitespace 
def remove_whitespace(text): 
    return  " ".join(text.split()) 

docs = df['text'].str.lower().values # make the words lower string
docs = [remove_punctuation(text) for text in docs]
docs = [remove_whitespace(text) for text in docs]

# split data into train and test set
test_data_size = .2 # use 20% data for testing, 80% for training

# get all the words for each language and save it in a list
word_list = [i.split() for i in  docs]

# generate training and test data for each language using a map fucntion
# we will split the words in each language into train and test
split_data = lambda x:  train_test_split(x,test_size=test_data_size)


# get a list which contains the test and train data splits
z = list(map(split_data, word_list))
# z2 = [train_test_split(x,test_size=.2,radnom_state=42) for x in word_list];

# index the train and test data from the list z
train_data = [item[0] for item in z]
test_data = [item[1] for item in z]

(21, 3)

str

list

In [273]:
get_language_id(1,train_data,test_data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [0]:
# this function performs language identification given train and test data and the ngram number 
# train_data, test_data contain a list for each language

def get_language_id(ngram_number,train_data,test_data):
    
  # perform statistics on training data for all languages
  # define function to get unigram frequencies 
  get_frequency_unigram = lambda x:  nltk.FreqDist((ngrams(' '.join(map(str, x)), ngram_number)))

  # get unigram counts for train and test data
  list_unigram_train = list(map(get_frequency_unigram, train_data))
  list_unigram_test = list(map(get_frequency_unigram, test_data))

  # make dataframe
  train_data_df = pd.DataFrame(list_unigram_train,index=df['language'])
  test_data_df = pd.DataFrame(list_unigram_test,index=df['language'])

  # a count of 0.5 is added to events seen in at least
  # one language in the training set
  np.max(train_data_df.isna().sum())

  # all colums have atleast one nonempty values given how python constructs dictionaries
  # thus we replace all NAs with .5
  train_data_df = train_data_df.fillna(.5)

  # get probabilities from counts by dividing the sum
  train_data_df = train_data_df.div(train_data_df.sum(axis=1), axis=0)
  test_data_df = test_data_df.div(test_data_df.sum(axis=1), axis=0)

  # get common ngram columns between train and test data
  common_cols =  np.intersect1d(train_data_df.columns, test_data_df.columns)
  len(common_cols)

  train_data_df2 = train_data_df[common_cols]
  test_data_df2 = test_data_df[common_cols]

  # perform language indetification using relative entropy
  for ii in range(0,len(list_unigram_test)):

    # compute the KL distance 
    D = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)

    # find index of the train language which has the minimum score (closest language in the train set)
    index = np.argmin(D)

    print(index)